In [2]:
import itertools
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import multiprocessing as mp
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import warnings
import logging
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3
from scipy.stats import kendalltau

# Suppress all warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('xgboost').setLevel(logging.ERROR)
logging.getLogger('lightgbm').setLevel(logging.ERROR)

class SimulationConfig:
    def __init__(self):
        self.n_simulations_options = [100]  # Fixed number of simulations
        self.sample_sizes = [1000, 2500]  # Sample sizes
        self.n_features_options = [7, 10]  # Feature counts to compare
        self.friedman_types = ["friedman1"]  # Using only one type for simplicity
        # Define meta-models for different methods
        self.meta_models = {
            'method1': LinearRegression(),
            'method2': Ridge(),
            'method3': Ridge(),
            'method4': Ridge()
        }

def generate_friedman_data(n_samples, n_features, friedman_type, random_state=None):
    """Generate Friedman dataset based on type"""
    if friedman_type == "friedman1":
        X, y = make_friedman1(n_samples=n_samples, n_features=n_features, random_state=random_state)

    else:
        raise ValueError(f"Unknown Friedman type: {friedman_type}")
    
    return X, y

def compute_stacking_feature_importance_method1(X, y, X_test, y_test, base_models, meta_model):
    """Compute feature importance for stacking model - Method 1 (Linear Regression)"""
    # Generate predictions from base models
    base_train_preds = np.column_stack([
        model.predict(X) for model in base_models
    ])

    base_test_preds = np.column_stack([
        model.predict(X_test) for model in base_models
    ])

    # Train meta-model
    meta_model.fit(base_train_preds, y)

    # Get meta-model weights
    if hasattr(meta_model, 'coef_'):
        meta_model_weights = np.abs(meta_model.coef_)
    else:
        meta_model_weights = np.ones(len(base_models)) / len(base_models)
    meta_model_weights = meta_model_weights / np.sum(meta_model_weights)

    # Get feature importance from base models
    base_importances = []
    for model in base_models:
        importance = model.feature_importances_
        importance = importance / np.sum(importance)
        base_importances.append(importance)
    base_importances = np.array(base_importances)

    # Calculate final feature importance
    overall_importance = np.zeros(X.shape[1])
    for i, importance in enumerate(base_importances):
        overall_importance += importance * meta_model_weights[i]

    # Ensure normalization
    overall_importance = overall_importance / np.sum(overall_importance)
    
    # Get predictions
    stacking_pred = meta_model.predict(base_test_preds)

    return overall_importance, meta_model_weights, meta_model, stacking_pred

def compute_stacking_feature_importance_method2(X, y, X_test, y_test, base_models, meta_model):
    """Compute feature importance for stacking model - Method 2 (Ridge Regression)"""
    # Generate predictions from base models
    base_train_preds = np.column_stack([
        model.predict(X) for model in base_models
    ])

    base_test_preds = np.column_stack([
        model.predict(X_test) for model in base_models
    ])

    # Train meta-model
    meta_model.fit(base_train_preds, y)

    # Get meta-model weights
    if hasattr(meta_model, 'coef_'):
        meta_model_weights = np.abs(meta_model.coef_)
    else:
        meta_model_weights = np.ones(len(base_models)) / len(base_models)
    meta_model_weights = meta_model_weights / np.sum(meta_model_weights)

    # Get feature importance from base models
    base_importances = []
    for model in base_models:
        importance = model.feature_importances_
        importance = importance / np.sum(importance)
        base_importances.append(importance)
    base_importances = np.array(base_importances)

    # Calculate final feature importance
    overall_importance = np.zeros(X.shape[1])
    for i, importance in enumerate(base_importances):
        overall_importance += importance * meta_model_weights[i]

    # Ensure normalization
    overall_importance = overall_importance / np.sum(overall_importance)
    
    # Get predictions
    stacking_pred = meta_model.predict(base_test_preds)

    return overall_importance, meta_model_weights, meta_model, stacking_pred

def compute_stacking_feature_importance_method3(X, y, X_test, y_test, base_models, meta_model):
    """Compute feature importance for stacking model with stability adjustment"""
    # Generate predictions from base models
    base_train_preds = np.column_stack([
        model.predict(X) for model in base_models
    ])

    base_test_preds = np.column_stack([
        model.predict(X_test) for model in base_models
    ])

    # Train meta-model
    meta_model.fit(base_train_preds, y)

    # Get meta-model weights
    if hasattr(meta_model, 'coef_'):
        meta_model_weights = np.abs(meta_model.coef_)
    else:
        meta_model_weights = np.ones(len(base_models)) / len(base_models)
    meta_model_weights = meta_model_weights / np.sum(meta_model_weights)

    # Get feature importance from base models
    base_importances = []
    for model in base_models:
        importance = model.feature_importances_
        importance = importance / np.sum(importance)
        base_importances.append(importance)
    base_importances = np.array(base_importances)

    # Calculate stability adjustment
    importance_std = np.std(base_importances, axis=0)  # Standard deviation across models for each feature
    epsilon = 1  # Small constant to avoid division by zero
    stability_adjustment = 1 / (importance_std + epsilon)

    # Normalize stability adjustment
    stability_adjustment = stability_adjustment / np.sum(stability_adjustment)

    # Calculate final feature importance
    overall_importance = np.zeros(X.shape[1])
    for i, importance in enumerate(base_importances):
        overall_importance += importance * meta_model_weights[i]

    # Apply stability adjustment to overall importance
    adjusted_importance = overall_importance * stability_adjustment

    # Ensure normalization
    adjusted_importance = adjusted_importance / np.sum(adjusted_importance)
    
    # Get predictions
    stacking_pred = meta_model.predict(base_test_preds)

    return adjusted_importance, meta_model_weights, meta_model, stacking_pred

def compute_stacking_feature_importance_method4(X, y, X_test, y_test, base_models, meta_model):
    """Compute feature importance for stacking model with Kendall's Tau penalty"""
    # Generate predictions from base models
    base_train_preds = np.column_stack([
        model.predict(X) for model in base_models
    ])

    base_test_preds = np.column_stack([
        model.predict(X_test) for model in base_models
    ])

    # Train meta-model
    meta_model.fit(base_train_preds, y)

    # Get meta-model weights
    if hasattr(meta_model, 'coef_'):
        meta_model_weights = np.abs(meta_model.coef_)
    else:
        meta_model_weights = np.ones(len(base_models)) / len(base_models)
    meta_model_weights = meta_model_weights / np.sum(meta_model_weights)

    # Get feature importance from base models
    base_importances = []
    for model in base_models:
        importance = model.feature_importances_
        importance = importance / np.sum(importance)
        base_importances.append(importance)
    base_importances = np.array(base_importances)

    # Calculate Kendall's Tau for each pair of models to assess ranking consistency
    n_models = len(base_models)
    tau_matrix = np.zeros((n_models, n_models))
    
    for i in range(n_models):
        for j in range(i+1, n_models):
            # Calculate Kendall's Tau for feature importance rankings between models
            tau, _ = kendalltau(base_importances[i], base_importances[j])
            # Convert from [-1,1] range to [0,1] range where 1 means perfect agreement
            tau_norm = (tau + 1) / 2
            tau_matrix[i, j] = tau_norm
            tau_matrix[j, i] = tau_norm
    
    # Calculate average Kendall's Tau for each model (row-wise mean)
    avg_tau = np.mean(tau_matrix, axis=1)
    
    # Convert average Tau to penalty factor (higher tau → higher penalty)
    # Use inverse relationship: 2 - tau to make higher tau values have higher penalties
    tau_penalty = 2 - avg_tau
    
    # Apply tau penalty to meta model weights
    adjusted_meta_weights = meta_model_weights * tau_penalty
    adjusted_meta_weights = adjusted_meta_weights / np.sum(adjusted_meta_weights)

    # Calculate final feature importance using penalized weights
    overall_importance = np.zeros(X.shape[1])
    for i, importance in enumerate(base_importances):
        overall_importance += importance * adjusted_meta_weights[i]

    # Ensure normalization of final importance values
    adjusted_importance = overall_importance / np.sum(overall_importance)
    
    # Get predictions
    stacking_pred = meta_model.predict(base_test_preds)

    return adjusted_importance, adjusted_meta_weights, meta_model, stacking_pred

def run_simulation(n_simulations, n_samples, n_features, friedman_type, config):
    """Run simulation with given parameters"""
    # Store results for each method
    methods_fi_results = {
        'method1': np.zeros((n_simulations, n_features)),
        'method2': np.zeros((n_simulations, n_features)),
        'method3': np.zeros((n_simulations, n_features)),
        'method4': np.zeros((n_simulations, n_features))
    }
    
    # Store meta model weights
    methods_meta_weights = {
        'method1': np.zeros((n_simulations, 6)),
        'method2': np.zeros((n_simulations, 6)),
        'method3': np.zeros((n_simulations, 6)),
        'method4': np.zeros((n_simulations, 6))
    }
    
    # Store model performance metrics
    methods_scores = {
        'method1': {'r2': [], 'mse': []},
        'method2': {'r2': [], 'mse': []},
        'method3': {'r2': [], 'mse': []},
        'method4': {'r2': [], 'mse': []}
    }

    for i in range(n_simulations):
        # Generate data from Friedman dataset
        X, y = generate_friedman_data(n_samples, n_features, friedman_type, random_state=i)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Initialize base models
        base_models = [
            xgb.XGBRegressor(random_state=i),
            lgb.LGBMRegressor(random_state=i, verbose=-1),
            RandomForestRegressor(random_state=i),
            ExtraTreesRegressor(random_state=i),
            DecisionTreeRegressor(random_state=i),
            AdaBoostRegressor(random_state=i)
        ]
        
        # Train base models
        for model in base_models:
            model.fit(X_train, y_train)
        
        # Define methods with their specific feature importance and meta-model
        methods = {
            'method1': (compute_stacking_feature_importance_method1, config.meta_models['method1']),
            'method2': (compute_stacking_feature_importance_method2, config.meta_models['method2']),
            'method3': (compute_stacking_feature_importance_method3, config.meta_models['method3']),
            'method4': (compute_stacking_feature_importance_method4, config.meta_models['method4'])
        }
        
        # Compute feature importance for each method
        for method_name, (method_func, meta_model) in methods.items():
            # Compute stacking feature importance
            stacking_output = method_func(
                X_train, y_train, X_test, y_test, base_models, meta_model
            )
            
            # Store results
            methods_fi_results[method_name][i] = stacking_output[0]
            methods_meta_weights[method_name][i] = stacking_output[1]
            
            # Get stacking predictions and compute performance metrics
            stacking_pred = stacking_output[3]
            methods_scores[method_name]['r2'].append(r2_score(y_test, stacking_pred))
            methods_scores[method_name]['mse'].append(mean_squared_error(y_test, stacking_pred))

    return methods_fi_results, methods_scores

def plot_feature_importance_boxplots(methods_fi_results, feature_count, output_dir):
    """Create boxplots to compare feature importance calculation methods"""
    plt.figure(figsize=(15, 10))
    
    n_features = feature_count
    
    # Create subplot grid based on number of features
    n_cols = 2
    n_rows = (n_features + 1) // 2  # Ceiling division
    
    method_names = ['Method 1', 'Method 2', 'Method 3', 'Method 4']
    
    for feat_idx in range(n_features):
        plt.subplot(n_rows, n_cols, feat_idx + 1)
        
        # Extract importance values for this feature across all simulations for each method
        feat_importances = [methods_fi_results[method][:, feat_idx] for method in methods_fi_results]
        
        # Create boxplot
        plt.boxplot(feat_importances, labels=method_names)
        plt.title(f'Feature {feat_idx+1} Importance')
        plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.suptitle(f'Stacking Feature Importance Methods Comparison - {n_features} Features', y=1.02, fontsize=16)
    plt.tight_layout()
    plt.savefig(output_dir / f'stacking_importance_comparison_f{n_features}.png', dpi=300, bbox_inches='tight')
    plt.close()

def compare_importance_methods(methods_fi_results, feature_count, output_dir):
    """Compare different stacking feature importance methods"""
    # Calculate mean and standard deviation for each method
    mean_fi = {method: np.mean(results, axis=0) for method, results in methods_fi_results.items()}
    std_fi = {method: np.std(results, axis=0) for method, results in methods_fi_results.items()}
    
    method_names = list(methods_fi_results.keys())
    
    # Create bar chart for comparison
    plt.figure(figsize=(15, 6))
    
    x = np.arange(feature_count)
    width = 0.2
    
    # Plot bars for each method
    for i, method in enumerate(method_names):
        plt.bar(x + i*width - width*1.5, mean_fi[method], width, 
                label=f'Method {i+1}', alpha=0.7, 
                yerr=std_fi[method], capsize=5)
    
    plt.title(f'Feature Importance Methods Comparison - {feature_count} Features')
    plt.xlabel('Feature Index')
    plt.ylabel('Mean Importance')
    plt.xticks(x, [f'F{j+1}' for j in range(feature_count)])
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.savefig(output_dir / f'importance_comparison_f{feature_count}.png', dpi=300)
    plt.close()
    
    # Calculate correlation matrix between methods
    method_correlations = np.zeros((len(method_names), len(method_names)))
    for i, method1 in enumerate(method_names):
        for j, method2 in enumerate(method_names):
            method_correlations[i, j] = np.corrcoef(
                mean_fi[method1], mean_fi[method2]
            )[0, 1]
    
    # Create heatmap of method correlations
    plt.figure(figsize=(10, 8))
    sns.heatmap(method_correlations, annot=True, cmap='coolwarm', 
                xticklabels=method_names, yticklabels=method_names)
    plt.title('Correlation Between Feature Importance Methods')
    plt.tight_layout()
    plt.savefig(output_dir / 'method_correlations_heatmap.png', dpi=300)
    plt.close()
    
    return method_correlations

def analyze_stability(methods_fi_results, feature_count, output_dir):
    """Analyze stability of feature importance methods"""
    # Calculate coefficient of variation (CV) for each method
    cv_methods = {}
    for method, results in methods_fi_results.items():
        cv_methods[method] = np.std(results, axis=0) / np.mean(results, axis=0)
    
    # Create bar chart for stability comparison
    plt.figure(figsize=(12, 6))
    
    x = np.arange(feature_count)
    width = 0.2
    
    method_names = list(methods_fi_results.keys())
    
    # Plot CV for each method
    for i, method in enumerate(method_names):
        plt.bar(x + i*width - width*1.5, cv_methods[method], width, 
                label=f'Method {i+1}', alpha=0.7)
    
    plt.title(f'Stability Comparison (Lower CV = More Stable) - {feature_count} Features')
    plt.xlabel('Feature Index')
    plt.ylabel('Coefficient of Variation')
    plt.xticks(x, [f'F{j+1}' for j in range(feature_count)])
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.savefig(output_dir / f'stability_comparison_f{feature_count}.png', dpi=300)
    plt.close()
    
    # Calculate overall stability metrics
    mean_cv_methods = {method: np.mean(cv) for method, cv in cv_methods.items()}
    
    return {
        'cv_methods': cv_methods,
        'mean_cv_methods': mean_cv_methods
    }

def generate_summary_report(methods_fi_results, methods_scores, method_correlations, stability_metrics, output_dir):
    """Generate a summary report of the findings"""
    report_path = output_dir / 'summary_report.txt'
    
    with open(report_path, 'w') as f:
        f.write("Stacking Feature Importance Methods Comparison Report\n")
        f.write("=================================================\n\n")
        
        # Performance Metrics
        f.write("Model Performance Metrics:\n")
        f.write("----------------------\n")
        for method, scores in methods_scores.items():
            f.write(f"\n{method}:\n")
            # Calculate mean and std for R² and MSE
            r2_mean = np.mean(scores['r2'])
            r2_std = np.std(scores['r2'])
            mse_mean = np.mean(scores['mse'])
            mse_std = np.std(scores['mse'])
            
            f.write(f"  Mean R²: {r2_mean:.4f} (±{r2_std:.4f})\n")
            f.write(f"  Mean MSE: {mse_mean:.4f} (±{mse_std:.4f})\n")
        
        # Method Correlations
        f.write("\nFeature Importance Method Correlations:\n")
        f.write("------------------------------------\n")
        method_names = list(methods_fi_results.keys())
        for i, method1 in enumerate(method_names):
            for j, method2 in enumerate(method_names):
                if i != j:
                    f.write(f"{method1} vs {method2}: {method_correlations[i, j]:.4f}\n")
        
        # Stability Analysis
        f.write("\nMethod Stability Analysis:\n")
        f.write("------------------------\n")
        for method, mean_cv in stability_metrics['mean_cv_methods'].items():
            f.write(f"{method} Mean CV: {mean_cv:.4f}\n")
        
        # Recommendations
        f.write("\nRecommendations:\n")
        f.write("---------------\n")
        
        # Find most stable method
        most_stable_method = min(stability_metrics['mean_cv_methods'], 
                                 key=stability_metrics['mean_cv_methods'].get)
        
        f.write(f"Most stable method: {most_stable_method}\n")
        
        # Find highest performing method based on R²
        r2_means = {method: np.mean(scores['r2']) for method, scores in methods_scores.items()}
        best_method = max(r2_means, key=r2_means.get)
        
        f.write(f"Best performing method (R²): {best_method}\n")

def visualize_and_compare_methods(methods_fi_results, methods_scores):
    """Comprehensive visualization and comparison of methods"""
    # Create output directory
    output_dir = Path('feature_importance_comparison')
    output_dir.mkdir(exist_ok=True)
    
    # Get feature count from results
    feature_count = list(methods_fi_results.values())[0].shape[1]
    
    # Plot feature importance boxplots
    plot_feature_importance_boxplots(methods_fi_results, feature_count, output_dir)
    
    # Compare methods
    method_correlations = compare_importance_methods(methods_fi_results, feature_count, output_dir)
    
    # Analyze stability
    stability_metrics = analyze_stability(methods_fi_results, feature_count, output_dir)
    
    # Generate summary report
    generate_summary_report(methods_fi_results, methods_scores, 
                            method_correlations, stability_metrics, output_dir)


def run_all_scenarios():
    """Run all scenarios and generate comparisons"""
    # Initialize configuration
    config = SimulationConfig()
    
    # Get all possible scenarios
    scenarios = list(itertools.product(
        config.n_simulations_options,
        config.sample_sizes,
        config.n_features_options,
        config.friedman_types
    ))
    
    print(f"Running {len(scenarios)} scenarios...")
    
    # Set up parallel processing
    n_processes = max(1, mp.cpu_count() - 1)
    pool = mp.Pool(processes=n_processes)
    
    # Run scenarios in parallel
    results = []
    
    try:
        for i, scenario in enumerate(scenarios):
            n_sims, sample_size, n_features, friedman_type = scenario
            
            try:
                # Run simulation for all 4 methods
                methods_fi_results, methods_scores = run_simulation(
                    n_simulations=n_sims,
                    n_samples=sample_size,
                    n_features=n_features,
                    friedman_type=friedman_type,
                    config=config
                )
                
                # Prepare result dictionary
                result = {
                    'success': True,
                    'n_simulations': n_sims,
                    'sample_size': sample_size,
                    'n_features': n_features,
                    'friedman_type': friedman_type,
                    'methods_fi_results': methods_fi_results,
                    'methods_scores': methods_scores
                }
                
                results.append(result)
                print(f"Completed scenario {i+1}/{len(scenarios)}: {n_features} features, {sample_size} samples")
            
            except Exception as e:
                print(f"Failed scenario {i+1}/{len(scenarios)}: {str(e)}")
                results.append({
                    'success': False,
                    'n_simulations': n_sims,
                    'sample_size': sample_size,
                    'n_features': n_features,
                    'friedman_type': friedman_type,
                    'error': str(e)
                })
    
    except KeyboardInterrupt:
        print("\nReceived keyboard interrupt, stopping gracefully...")
        pool.terminate()
    finally:
        pool.close()
        pool.join()
    
    # Generate comparison visualizations
    successful_results = [r for r in results if r.get('success', False)]
    
    if successful_results:
        print("Generating feature importance comparisons...")
        
        # Group results by feature count
        feature_count_groups = {}
        for scenario in successful_results:
            n_features = scenario['n_features']
            if n_features not in feature_count_groups:
                feature_count_groups[n_features] = []
            feature_count_groups[n_features].append(scenario)
        
        # Process each feature count group separately
        for n_features, scenarios_group in feature_count_groups.items():
            # Aggregate results for this feature count
            all_methods_fi_results = {}
            all_methods_scores = {}
            
            # Collect results for each method across scenarios with this feature count
            method_keys = ['method1', 'method2', 'method3', 'method4']
            for method in method_keys:
                # Collect feature importance results for this method and feature count
                method_fi_results = [
                    scenario['methods_fi_results'][method] 
                    for scenario in scenarios_group
                ]
                
                # Ensure all arrays have the same number of features
                method_fi_results = [
                    arr[:, :n_features] if arr.shape[1] > n_features else arr 
                    for arr in method_fi_results
                ]
                
                # Combine results
                all_methods_fi_results[method] = np.vstack(method_fi_results)
                
                # Combine scores
                all_methods_scores[method] = {
                    'r2': [score for scenario in scenarios_group 
                           for score in scenario['methods_scores'][method]['r2']],
                    'mse': [score for scenario in scenarios_group 
                            for score in scenario['methods_scores'][method]['mse']]
                }
            
            # Visualize and compare methods for this feature count
            visualize_and_compare_methods(all_methods_fi_results, all_methods_scores)
        
        print("Comparison generation complete.")
    else:
        print("No successful scenarios to summarize.")

# Main execution
if __name__ == '__main__':
    try:
        # Run all scenarios and generate comparisons
        run_all_scenarios()
        print("Analysis completed. Results are in the 'feature_importance_comparison' directory.")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()

Running 4 scenarios...

Received keyboard interrupt, stopping gracefully...
No successful scenarios to summarize.
Analysis completed. Results are in the 'feature_importance_comparison' directory.
